In [109]:
import time
import datetime
from collections import defaultdict

import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [110]:
features = pd.read_csv('features.csv', index_col='match_id')
features.drop(
    ['start_time', 'duration', 'tower_status_radiant', 'tower_status_dire', 
     'barracks_status_dire', 'barracks_status_radiant'],
    axis=1,
    inplace=True
)
matches = features.shape[0]

Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [111]:
counts = features.count()
missing = counts[counts != matches]
for name in missing.index:
    print(name)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [112]:
for name in missing.index:
    features[name].fillna(0, inplace=True)

Какой столбец содержит целевую переменную? Запишите его название.

In [113]:
y = features['radiant_win']
features.drop('radiant_win', axis=1, inplace=True)

X = features.copy()

Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [114]:
kf = KFold(n_splits=5, shuffle=True)
scores = defaultdict(list)
for n_estimators in (10, 20, 30):
    clf = GradientBoostingClassifier(n_estimators=n_estimators)
    for train_index, test_index in kf.split(X, y):
        start_time = datetime.datetime.now()
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        clf.fit(X_train, y_train)
        
        probs = clf.predict_proba(X_test)
        
        print('Time elapsed for n_estimaters={}: {}'.format(n_estimators, datetime.datetime.now() - start_time))
        
        scores[n_estimators].append(roc_auc_score(y_test, probs[:, 1]))
    scores[n_estimators] = np.mean(scores[n_estimators])

Time elapsed for n_estimaters=10: 0:00:07.524770
Time elapsed for n_estimaters=10: 0:00:07.718111
Time elapsed for n_estimaters=10: 0:00:07.569763
Time elapsed for n_estimaters=10: 0:00:07.467916
Time elapsed for n_estimaters=10: 0:00:07.993642
Time elapsed for n_estimaters=20: 0:00:12.774479
Time elapsed for n_estimaters=20: 0:00:13.120844
Time elapsed for n_estimaters=20: 0:00:12.451629
Time elapsed for n_estimaters=20: 0:00:12.936708
Time elapsed for n_estimaters=20: 0:00:11.951211
Time elapsed for n_estimaters=30: 0:00:20.319237
Time elapsed for n_estimaters=30: 0:00:19.426570
Time elapsed for n_estimaters=30: 0:00:19.691805
Time elapsed for n_estimaters=30: 0:00:18.293860
Time elapsed for n_estimaters=30: 0:00:18.350798


In [115]:
scores

defaultdict(list,
            {10: 0.66480789207044266,
             20: 0.68200053631530844,
             30: 0.6898704609090146})

Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [7]:
def score_logistic_regression(X, y):
    best_score = 0
    best_c = 0
    kf = KFold(n_splits=5, shuffle=True)
    for C in np.logspace(-3, 0, num=5):
        clf = LogisticRegression(penalty='l2', C=C)
        for train_index, test_index in kf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
            start_time = datetime.datetime.now()
            clf.fit(X_train, y_train)
            probs = clf.predict_proba(X_test)
            print('Time elapsed for C={}: {}'.format(C, datetime.datetime.now() - start_time))
        
            score = roc_auc_score(y_test, probs[:, 1])
            if score > best_score:
                best_score = score
                best_c = C
    return best_score, best_c

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
best_score, best_c = score_logistic_regression(X_scaled, y)

Time elapsed for C=0.001: 0:00:01.390618
Time elapsed for C=0.001: 0:00:01.710314
Time elapsed for C=0.001: 0:00:01.725432
Time elapsed for C=0.001: 0:00:01.446161
Time elapsed for C=0.001: 0:00:01.470715
Time elapsed for C=0.005623413251903491: 0:00:02.757079
Time elapsed for C=0.005623413251903491: 0:00:02.301870
Time elapsed for C=0.005623413251903491: 0:00:02.183077
Time elapsed for C=0.005623413251903491: 0:00:02.242043
Time elapsed for C=0.005623413251903491: 0:00:02.110328
Time elapsed for C=0.03162277660168379: 0:00:02.335798
Time elapsed for C=0.03162277660168379: 0:00:02.428356
Time elapsed for C=0.03162277660168379: 0:00:02.480551
Time elapsed for C=0.03162277660168379: 0:00:02.343630
Time elapsed for C=0.03162277660168379: 0:00:02.391013
Time elapsed for C=0.1778279410038923: 0:00:02.599273
Time elapsed for C=0.1778279410038923: 0:00:02.378384
Time elapsed for C=0.1778279410038923: 0:00:02.544275
Time elapsed for C=0.1778279410038923: 0:00:02.526753
Time elapsed for C=0.177

In [10]:
best_score, best_c

(0.72315131659748866, 0.031622776601683791)

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [11]:
categorial_features = [
    'lobby_type',
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero'
]

In [12]:
X = features.drop(categorial_features, axis=1)

In [13]:
X_scaled = scaler.fit_transform(X)

In [14]:
best_score, best_c = score_logistic_regression(X_scaled, y)

Time elapsed for C=0.001: 0:00:01.250531
Time elapsed for C=0.001: 0:00:02.006933
Time elapsed for C=0.001: 0:00:01.970099
Time elapsed for C=0.001: 0:00:01.590641
Time elapsed for C=0.001: 0:00:01.232115
Time elapsed for C=0.005623413251903491: 0:00:02.180531
Time elapsed for C=0.005623413251903491: 0:00:01.980709
Time elapsed for C=0.005623413251903491: 0:00:01.941825
Time elapsed for C=0.005623413251903491: 0:00:01.961808
Time elapsed for C=0.005623413251903491: 0:00:01.721294
Time elapsed for C=0.03162277660168379: 0:00:03.170221
Time elapsed for C=0.03162277660168379: 0:00:03.096205
Time elapsed for C=0.03162277660168379: 0:00:02.378050
Time elapsed for C=0.03162277660168379: 0:00:02.354792
Time elapsed for C=0.03162277660168379: 0:00:02.274782
Time elapsed for C=0.1778279410038923: 0:00:02.545826
Time elapsed for C=0.1778279410038923: 0:00:02.200557
Time elapsed for C=0.1778279410038923: 0:00:02.262658
Time elapsed for C=0.1778279410038923: 0:00:02.108043
Time elapsed for C=0.177

In [15]:
best_score, best_c

(0.7211168571078661, 0.005623413251903491)

На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [16]:
heroes = categorial_features[:]
heroes.remove('lobby_type')
unique_heroes = set()
for hero_feature_name in heroes:
    unique_heroes.update(features[hero_feature_name].unique())
print(len(unique_heroes))
unique_heroes = list(unique_heroes)

108


Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [17]:
X_pick = np.zeros((features.shape[0], 108))
for i, match_id in enumerate(features.index):
    for p in range(5):
        r_hero_code = features.ix[match_id, 'r%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(r_hero_code)] = 1
        
        d_hero_code = features.ix[match_id, 'd%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(d_hero_code)] = -1

In [18]:
X = np.hstack((X, X_pick))
X_scaled = scaler.fit_transform(X)

Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [19]:
best_score, best_c = score_logistic_regression(X_scaled, y)

Time elapsed for C=0.001: 0:00:03.195017
Time elapsed for C=0.001: 0:00:03.284971
Time elapsed for C=0.001: 0:00:02.852921
Time elapsed for C=0.001: 0:00:02.856956
Time elapsed for C=0.001: 0:00:02.978540
Time elapsed for C=0.005623413251903491: 0:00:03.755062
Time elapsed for C=0.005623413251903491: 0:00:03.748924
Time elapsed for C=0.005623413251903491: 0:00:03.654423
Time elapsed for C=0.005623413251903491: 0:00:03.894805
Time elapsed for C=0.005623413251903491: 0:00:03.644014
Time elapsed for C=0.03162277660168379: 0:00:03.961687
Time elapsed for C=0.03162277660168379: 0:00:04.510821
Time elapsed for C=0.03162277660168379: 0:00:04.898055
Time elapsed for C=0.03162277660168379: 0:00:04.282829
Time elapsed for C=0.03162277660168379: 0:00:04.018072
Time elapsed for C=0.1778279410038923: 0:00:04.122521
Time elapsed for C=0.1778279410038923: 0:00:04.350694
Time elapsed for C=0.1778279410038923: 0:00:04.718328
Time elapsed for C=0.1778279410038923: 0:00:04.379893
Time elapsed for C=0.177

In [20]:
best_score, best_c # после добавления `мешка слов` качество улучшилось

(0.75781959002067301, 1.0)

Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [46]:
X_test = pd.read_csv('features_test.csv', index_col='match_id')
X_test.drop([
    'start_time',
    'lobby_type'
], axis=1, inplace=True)
matches = X_test.shape[0]

In [47]:
counts = X_test.count()
missing = counts[counts != matches]
for name in missing.index:
    X_test[name].fillna(0, inplace=True)

In [48]:
X_pick = np.zeros((X_test.shape[0], 108))
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        r_hero_code = X_test.ix[match_id, 'r%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(r_hero_code)] = 1
        
        d_hero_code = X_test.ix[match_id, 'd%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(d_hero_code)] = -1

In [54]:
X_test.drop([
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero'
], axis=1, inplace=True)

In [56]:
X_merged = np.hstack((X_test, X_pick))

In [57]:
X_merged_scaled = scaler.fit_transform(X_merged)

In [31]:
clf = LogisticRegression(penalty='l2', C=1.0)

In [32]:
clf.fit(X_scaled, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [58]:
probs = clf.predict_proba(X_merged_scaled)[:, 1]

In [59]:
probs

array([ 0.83633895,  0.77901248,  0.20100047, ...,  0.22564408,
        0.62264571,  0.41819582])

In [62]:
probs.shape

(17177,)

In [94]:
res = {'match_id': [], 'radiant_win': []}
for i, match_id in enumerate(X_test.index):
    res['match_id'].append(match_id)
    res['radiant_win'].append(probs[i])

In [106]:
res_df = pd.DataFrame(res)
res_df.set_index('match_id', inplace=True)

In [107]:
res_df.to_csv('final-results.csv')